In [67]:
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
from pymongo import MongoClient
import pandas as pd
from pandas.io.json import json_normalize

In [68]:
start_lat = 47.378611
start_lon = 8.540000
heat_m = folium.Map(location=[start_lat, start_lon],tiles='cartodbpositron', zoom_start=15)
heat_m

In [69]:

uni_lon, uni_lat = 8.540000, 47.378611
Marker([uni_lat, uni_lon], icon=folium.Icon(color='red')).add_to(heat_m)
heat_m

## Añadimos al mapa las empresas de Zúrich

In [70]:

client = MongoClient("mongodb://localhost/companies")
db = client.get_database()
cursor = db["cleancompanies"].find({'$and':[{ 'latitude' : { '$exists': True, '$ne':None} },{'city':'Zurich'}]})
zurcomp = pd.DataFrame(cursor)
zurcomp

,_id,name,category_code,total_money_raised,founded_year,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,location
0,5e418a34b2f9d04bed1a5389,Amazee,public_relations,$0,2007.0,World Headquarters,Technoparkstr. 1,Newton 2012,8005,Zurich,None,CHE,47.389909,8.516049,"{'type': 'Point', 'coordinates': [8.516049, 47..."
1,5e418a34b2f9d04bed1a543c,Cassiber,web,$0,2007.0,,Technoparkstrasse 1,,8005,Zurich,None,CHE,47.389909,8.516049,"{'type': 'Point', 'coordinates': [8.516049, 47..."
2,5e418a34b2f9d04bed1a54b5,Wuala,software,$0,2007.0,,Köchlistrasse 15,,8004,Zurich,None,CHE,47.384199,8.492452,"{'type': 'Point', 'coordinates': [8.492452, 47..."
3,5e418a34b2f9d04bed1a550c,GenePartner,biotech,$0,2008.0,,Schlossgasse 9,,8003,Zurich,None,CHE,47.368986,8.519982,"{'type': 'Point', 'coordinates': [8.519982, 47..."
4,5e418a34b2f9d04bed1a5675,Doodle,web,$0,2007.0,Headquarters,Werdstr. 21,,8021,Zurich,None,CHE,47.389909,8.516049,"{'type': 'Point', 'coordinates': [8.516049, 47..."
5,5e418a34b2f9d04bed1a5705,GetYourGuide,travel,$16M,2009.0,Headquarters,GetYourGuide AG,Technoparkstr. 1,8005,Zurich,None,CHE,47.391290,8.508289,"{'type': 'Point', 'coordinates': [8.5082891, 4..."
6,5e418a34b2f9d04bed1a60e9,Smaboo,other,$0,2007.0,Headquarter,Lindenstr. 41,,8008,Zurich,None,CHE,47.358286,8.553588,"{'type': 'Point', 'coordinates': [8.553588, 47..."
7,5e418a34b2f9d04bed1a7836,NearSoft Europe,software,$0,2006.0,NearSoft Europe,Sumatrastrasse 1,,8006,Zurich,None,CHE,47.379943,8.543685,"{'type': 'Point', 'coordinates': [8.5436847, 4..."
8,5e418a34b2f9d04bed1a837d,SVOX,mobile,$0,NaN,,Baslerstrasse 30,,8048,Zurich,None,CHE,47.384988,8.502533,"{'type': 'Point', 'coordinates': [8.5025329, 4..."
9,5e418a34b2f9d04bed1a8646,Ergon Informatik,software,$0,1984.0,,Kleinstrasse 15,,8008,Zurich,None,CHE,47.363955,8.553415,"{'type': 'Point', 'coordinates': [8.5534151, 4..."


In [71]:
empresas = folium.FeatureGroup(name="Empresas")

HeatMap(data=zurcomp[['latitude', 'longitude']],radius=15).add_to(empresas)
empresas.add_to(heat_m)

heat_m

## Añadimos el aeropuerto de Zúrich

In [72]:
cursor2 = db['Airports'].find({'$and':[{ 'Latitude' : { '$exists': True, '$ne':None} },{'City':'Zurich'},{'Type':'airport'}]})
zurAirp = pd.DataFrame(cursor2)
zurAirp


,_id,Name,City,Country,Code,Latitude,Longitude,Type,location
0,5e4176ba3c488f5b719db862,Zürich Airport,Zurich,Switzerland,ZRH,47.464699,8.54917,airport,"{'type': 'Point', 'coordinates': [8.5491695404..."


In [73]:
heat_m.add_child(Marker([zurAirp['Latitude'], zurAirp['Longitude']], icon=folium.Icon(color='green')))
heat_m

## Añadimos los Starbucks de la ciudad

In [74]:
cursor3 = db["Starbucks"].find({'City':'Zurich'})
zurSB = pd.DataFrame(cursor3)

In [75]:
starbucks = folium.FeatureGroup(name="Starbucks")
HeatMap(data=zurSB[['Latitude', 'Longitude']],radius=15).add_to(starbucks)
starbucks.add_to(heat_m)
folium.LayerControl(collapsed=False).add_to(heat_m)
heat_m

## Añadimos los restaurantes veganos

In [76]:
cursor4 = db["VeganRestaurants"].find({})
zurVegans = pd.DataFrame(cursor4)

In [77]:
vegans = folium.FeatureGroup(name="Vegan Restaurants")
HeatMap(data=zurVegans[['lat', 'lng']],radius=15).add_to(vegans)
vegans.add_to(heat_m)
folium.LayerControl(collapsed=False).add_to(heat_m)
heat_m

In [52]:
zurAirp['location']

0    {'type': 'Point', 'coordinates': [8.5491695404...
Name: location, dtype: object

In [64]:
#numero de empresas a 10km del aeropuerto
Airport = {
            "type":"Point",
            "coordinates":[8.5491695404,47.4646987915]
            }

def withGeoQuery(location,maxDistance=10000,minDistance=0,field="location"):
    return {
       field: {
         "$near": {
           "$geometry": location,
           "$maxDistance": maxDistance,
           "$minDistance": minDistance
         }
       }
    }

query = withGeoQuery(Airport)
comp = list(db["cleancompanies"].find(query))
comp


[{'_id': ObjectId('5e418a34b2f9d04bed1a4baa'),
  'name': 'mpw Technologies',
  'category_code': 'mobile',
  'total_money_raised': '$0',
  'founded_year': 2003.0,
  'description': None,
  'address1': 'Frohburgstr. 100',
  'address2': '',
  'zip_code': '8006',
  'city': 'Zuerich',
  'state_code': None,
  'country_code': 'CHE',
  'latitude': 47.392111,
  'longitude': 8.550481,
  'location': {'type': 'Point', 'coordinates': [8.550481, 47.392111]}},
 {'_id': ObjectId('5e418a34b2f9d04bed1a8d80'),
  'name': 'Purple Labs',
  'category_code': 'mobile',
  'total_money_raised': '$12.7M',
  'founded_year': 2001.0,
  'description': '',
  'address1': 'Lagerstrasse 14',
  'address2': '',
  'zip_code': '8600',
  'city': 'DÃ¼bendorf',
  'state_code': None,
  'country_code': 'CHE',
  'latitude': 47.4004994,
  'longitude': 8.6029472,
  'location': {'type': 'Point', 'coordinates': [8.6029472, 47.4004994]}},
 {'_id': ObjectId('5e418a34b2f9d04bed1a8d86'),
  'name': 'Myriad',
  'category_code': 'mobile',
  '